# <img src="uni-logo.png" alt="Logo UNI" width=100 hight=200 align="right">


<br><br><br>
<h1><font color="#1D65DD" size=5>Python desde cero</font></h1>



<h1><font color="#1D65DD" size=6>Pandas II</font></h1>

<br>
<div style="text-align: right">
<font color="#1D65DD" size=3>Yuri Coicca, M.Sc.</font><br>

</div>

<a id="indice"></a>
<h2><font color="#7F000E" size=5>Índice</font></h2>


* [5. Selección y ordenación](#section5)
    * [Consulta y selección](#section51)  
    * [Ordenación](#section52)
* [6. Operaciones sobre elementos](#section6)
    * [Operaciones básicas](#section61)
    * [Operaciones de transformación sobre elementos](#section62)   
 
* [7. Agregación](#section7)   
    * [Estadísticos descriptivos](#section71)
    * [Agregación de datos: <font face="monospace">apply() y agg()</font>](#section72)  
    * [Agregación de datos mediante ventana deslizante: <font face="monospace">rolling()</font> y <font face="monospace">expanding()</font>](#section73)  
* [8. Agrupamiento: <font face="monospace">groupby()</font>](#section8)
    * [Agregación: <font face="monospace">GroupBy.agg()</font>](#section81)
    * [Transformación: <font face="monospace">GroupBy.transform()</font>](#section82)
    * [<font face="monospace">GroupBy.apply()</font>](#section83)
    * [Filtrado: <font face="monospace">GroupBy.filter()</font>](#section84)
    * [Eficiencia](#section85)

In [73]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:98%}</style>"))
import warnings
warnings.simplefilter('ignore')

---

<a id="section5"></a>
# <font color="#7F000E"> 5. Selección y ordenación </font>
<br>

Para ilustrar esta sección, se utilizará uno de los _DataFrames_  usados la libreta anterior. Se utilizará el nombre como índice. 

In [2]:

import pandas as pd

# Lee el archivo, utiliza el campo Name como índice.
df_fifa = pd.read_csv('./data/fifa19.csv', index_col=0).set_index('Name')
# Columnas seleccionadas
sel_columns = ['ID', 'Age', 'Nationality', 'Overall', 'Potential','Club', 'Value', 'Wage', 'Position', 'Joined', 'Height', 'Weight', 'Release Clause']
# Selecciona filas y columnas
df_fifa = df_fifa[:200][sel_columns] 
# Confierte las columnas a un formato adecuado (Cada columna es una serie)
df_fifa['Weight'] = df_fifa['Weight'].map(lambda w: float(w[:-3])*0.453592)
df_fifa['Height'] = df_fifa['Height'].map(lambda h: float(h[0])*30.48 + float(h[2])*2.54) 
df_fifa['Value'] = df_fifa['Value'].map(lambda v: float(v[1:-1]))
df_fifa['Wage'] = df_fifa['Wage'].map(lambda v: float(v[1:-1]))
# Renombra unas columnas
df_fifa.rename(columns={'Value':'Value (M)', 'Wage':'Wage (K)'}, inplace=True)
# Muestra la cabecera
df_fifa.head()

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
L. Messi,158023,31,Argentina,94,94,FC Barcelona,110.5,565.0,RF,"Jul 1, 2004",170.18,72.121128,€226.5M
Cristiano Ronaldo,20801,33,Portugal,94,94,Juventus,77.0,405.0,ST,"Jul 10, 2018",187.96,83.007336,€127.1M
Neymar Jr,190871,26,Brazil,92,93,Paris Saint-Germain,118.5,290.0,LW,"Aug 3, 2017",175.26,68.038800,€228.1M
De Gea,193080,27,Spain,91,93,Manchester United,72.0,260.0,GK,"Jul 1, 2011",193.04,76.203456,€138.6M
K. De Bruyne,192985,27,Belgium,91,92,Manchester City,102.0,355.0,RCM,"Aug 30, 2015",154.94,69.853168,€196.4M


<div class="alert alert-block alert-danger">

<i class="fa fa-exclamation-triangle" aria-hidden="true"></i>
Verán que, si tratan de utilizar el conjunto de datos completo, se produce un fallo. Esto se debe a que alguno de los valores numéricos codificados no están bien en el conjunto, y al decodificarlo con las funciones _lambda_ no corresponde al formato esperado. Estas situaciones se tienen que manejar en la lectura. Se verán ejemplos tanto en los ejercicios como en un seminario dedicado a limpieza y preparación, pero en este punto es mejor no complicar más la cosa. 
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section51"></a> 
## <font color="#7F000E">Consulta y selección </font>
<br>

La aplicación de un operador booleano sobre una columna, que es un objeto `Series`, genera como resultado otro objeto de tipo `Series` cuyo índice es el mismo del `DataFrame`, y en el que los elementos correspoden al resultado de la operación.

Por ejemplo, el siguiente código comprueba si los jugadores tienen menos de 30 años.

In [3]:
consulta = df_fifa['Age']<30
consulta[:5]                           # Muestra los 5 primeros elementos.

Name
L. Messi             False
Cristiano Ronaldo    False
Neymar Jr             True
De Gea                True
K. De Bruyne          True
Name: Age, dtype: bool

Es posible utilizar indexación lógica en un `DataFrame` para devolver las filas de interés.

In [4]:
jovenes = df_fifa.loc[df_fifa['Age']<30]
#jovenes = df_fifa[df_fifa['Age']<30]
#jovenes = df_fifa.loc[df_fifa['Age']<30,['Nationality','Overall']]

jovenes.head()
#jovenes = df_fifa[df_fifa['Age']<30,['Nationality','Overall']] # No funciona!

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
Neymar Jr,190871,26,Brazil,92,93,Paris Saint-Germain,118.5,290.0,LW,"Aug 3, 2017",175.26,68.038800,€228.1M
De Gea,193080,27,Spain,91,93,Manchester United,72.0,260.0,GK,"Jul 1, 2011",193.04,76.203456,€138.6M
K. De Bruyne,192985,27,Belgium,91,92,Manchester City,102.0,355.0,RCM,"Aug 30, 2015",154.94,69.853168,€196.4M
E. Hazard,183277,27,Belgium,91,91,Chelsea,93.0,340.0,LF,"Jul 1, 2012",172.72,73.935496,€172.1M
J. Oblak,200389,25,Slovenia,90,93,Atlético Madrid,68.0,94.0,GK,"Jul 16, 2014",187.96,87.089664,€144.5M


Se pueden utilizar las operaciones entre `Series` para generar condiciones más complejas. 

In [5]:
jovenes_españoles = df_fifa[(df_fifa['Age']<30) & (df_fifa['Nationality']=='Spain')]
jovenes_españoles.head()

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
De Gea,193080,27,Spain,91,93,Manchester United,72.0,260.0,GK,"Jul 1, 2011",193.04,76.203456,€138.6M
Sergio Busquets,189511,29,Spain,89,89,FC Barcelona,51.5,315.0,CDM,"Sep 1, 2008",187.96,76.203456,€105.6M
Isco,197781,26,Spain,88,91,Real Madrid,73.5,315.0,LW,"Jul 3, 2013",175.26,78.925008,€156.2M
Jordi Alba,189332,29,Spain,87,87,FC Barcelona,38.0,250.0,LB,"Jul 1, 2012",170.18,68.038800,€77.9M
Thiago,189509,27,Spain,86,86,FC Bayern München,45.5,130.0,CM,"Jul 14, 2013",175.26,69.853168,€75.1M


<div class="alert alert-block alert-danger">

<i class="fa fa-exclamation-triangle" aria-hidden="true"></i>
Debido a que los operadores binarios tienen precedencia sobre las comparaciones, la siguiente expresión, en la que se han quitado los paréntesis de la expresión, devolvería un error.
</div>

In [6]:
#jovenes_españoles = df_fifa[df_fifa['Age']<30 & df_fifa['Nationality']=='Spain']

### <font color="#7F000E" face="monospace"> where() </font>

Las consultas pueden hacerse mediante  el método `where()` (como en las `Series`). En ese caso se devuelven __todas las filas__, pero aquellas que no cumplen la condición se reemplazan (la fila entera) con un valor (por defecto NaN) que puede ser un valor o el contenido de otra columna.

In [7]:
#jovenes = df_fifa.where(df_fifa['Age']<30)
jovenes = df_fifa.where(df_fifa['Age']<30, '---')
jovenes.head(5)

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
L. Messi,---,---,---,---,---,---,---,---,---,---,---,---,---
Cristiano Ronaldo,---,---,---,---,---,---,---,---,---,---,---,---,---
Neymar Jr,190871,26,Brazil,92,93,Paris Saint-Germain,118.5,290,LW,"Aug 3, 2017",175.26,68.0388,€228.1M
De Gea,193080,27,Spain,91,93,Manchester United,72,260,GK,"Jul 1, 2011",193.04,76.2035,€138.6M
K. De Bruyne,192985,27,Belgium,91,92,Manchester City,102,355,RCM,"Aug 30, 2015",154.94,69.8532,€196.4M


Del mismo modo que la indexación mediante valores booleanos, `where` admite condiciones obtenidas mediante operaciones booleanas entre series. 

In [8]:
jovenes_españoles = df_fifa.where((df_fifa['Age']<30) & (df_fifa['Nationality']=='Spain'),'---')
jovenes_españoles.head(5)

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
L. Messi,---,---,---,---,---,---,---,---,---,---,---,---,---
Cristiano Ronaldo,---,---,---,---,---,---,---,---,---,---,---,---,---
Neymar Jr,---,---,---,---,---,---,---,---,---,---,---,---,---
De Gea,193080,27,Spain,91,93,Manchester United,72,260,GK,"Jul 1, 2011",193.04,76.2035,€138.6M
K. De Bruyne,---,---,---,---,---,---,---,---,---,---,---,---,---


### <font color="#7F000E" face="monospace"> mask() </font>

La función `mask` es opuesta a `where`, es decir, sustituye (_enmascara_) los valores que cumplen la condición.

In [9]:
nojovenes_noespañoles = df_fifa.mask((df_fifa['Age']<30) & (df_fifa['Nationality']=='Spain'),'---')
nojovenes_noespañoles.head(5)

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
L. Messi,158023,31,Argentina,94,94,FC Barcelona,110.5,565,RF,"Jul 1, 2004",170.18,72.1211,€226.5M
Cristiano Ronaldo,20801,33,Portugal,94,94,Juventus,77,405,ST,"Jul 10, 2018",187.96,83.0073,€127.1M
Neymar Jr,190871,26,Brazil,92,93,Paris Saint-Germain,118.5,290,LW,"Aug 3, 2017",175.26,68.0388,€228.1M
De Gea,---,---,---,---,---,---,---,---,---,---,---,---,---
K. De Bruyne,192985,27,Belgium,91,92,Manchester City,102,355,RCM,"Aug 30, 2015",154.94,69.8532,€196.4M


### <font color="#7F000E" face="monospace"> isin() </font>

La función `isin` también es de __suma utilidad__, ya que permite seleccionar las filas cuyo valor esté dentro de un conjunto. 

In [10]:
df_fifa[df_fifa['Nationality'].isin(['Spain','Portugal'])]

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
Cristiano Ronaldo,20801,33,Portugal,94,94,Juventus,77.0,405.0,ST,"Jul 10, 2018",187.96,83.007336,€127.1M
De Gea,193080,27,Spain,91,93,Manchester United,72.0,260.0,GK,"Jul 1, 2011",193.04,76.203456,€138.6M
Sergio Ramos,155862,32,Spain,91,91,Real Madrid,51.0,380.0,RCB,"Aug 1, 2005",182.88,82.100152,€104.6M
David Silva,168542,32,Spain,90,90,Manchester City,60.0,285.0,LCM,"Jul 14, 2010",172.72,67.131616,€111M
Sergio Busquets,189511,29,Spain,89,89,FC Barcelona,51.5,315.0,CDM,"Sep 1, 2008",187.96,76.203456,€105.6M
Isco,197781,26,Spain,88,91,Real Madrid,73.5,315.0,LW,"Jul 3, 2013",175.26,78.925008,€156.2M
Jordi Alba,189332,29,Spain,87,87,FC Barcelona,38.0,250.0,LB,"Jul 1, 2012",170.18,68.038800,€77.9M
Piqué,152729,31,Spain,87,87,FC Barcelona,34.0,240.0,RCB,"Jul 1, 2008",193.04,84.821704,€69.7M
Bernardo Silva,218667,23,Portugal,86,91,Manchester City,59.5,180.0,RW,"Jul 1, 2017",172.72,63.956472,€114.5M


La función `isin()` admite también que se especifiquen valores para cada columna mediante un diccionario. En ese caso devuelve un `DataFrame`, que ha de ser filtrado adecuadamente.

In [11]:
cond = {'Nationality':['Spain','Portugal','Argentina'], 'Club':['FC Barcelona','Real Madrid']} 
#df_fifa.isin(cond)[['Nationality','Club']]
df_fifa[df_fifa.isin(cond)[['Nationality','Club']].all(axis=1)]

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
L. Messi,158023,31,Argentina,94,94,FC Barcelona,110.5,565.0,RF,"Jul 1, 2004",170.18,72.121128,€226.5M
Sergio Ramos,155862,32,Spain,91,91,Real Madrid,51.0,380.0,RCB,"Aug 1, 2005",182.88,82.100152,€104.6M
Sergio Busquets,189511,29,Spain,89,89,FC Barcelona,51.5,315.0,CDM,"Sep 1, 2008",187.96,76.203456,€105.6M
Isco,197781,26,Spain,88,91,Real Madrid,73.5,315.0,LW,"Jul 3, 2013",175.26,78.925008,€156.2M
Jordi Alba,189332,29,Spain,87,87,FC Barcelona,38.0,250.0,LB,"Jul 1, 2012",170.18,68.038800,€77.9M
Piqué,152729,31,Spain,87,87,FC Barcelona,34.0,240.0,RCB,"Jul 1, 2008",193.04,84.821704,€69.7M
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M
Carvajal,204963,26,Spain,84,87,Real Madrid,31.5,185.0,RB,"Jul 5, 2013",172.72,73.028312,€66.9M
Lucas Vázquez,208618,27,Spain,83,83,Real Madrid,27.0,205.0,RW,"Jul 2, 2015",172.72,69.853168,€55.4M


### <font color="#7F000E" face="monospace"> filter() </font>

La función `filter()` permite filtrar elementos del `DataFrame` en función del valor del índice. Admite varios parámetros.

In [12]:
#df_fifa.filter(items=['Age','Nationality'], axis=1) # Columnas
#df_fifa.filter(like='De', axis=0)                   # En alguna parte del nombre aparece `De`
#df_fifa.filter(regex='\w* \w* \w*', axis=0)          # Expresión regular. Nombre con tres partes.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section52"></a> 
## <font color="#7F000E">Ordenación </font>
<br>

La operación `DataFrame.sort_index` ordena el `DataFrame` en función de su índice. Mediante el parámetro `ascending` se establece el orden de la ordenación. Además, por defecto produce un nuevo `DataFrame`, a no ser que se indique lo contrario mediante el parámetro `inplace`.

In [13]:
df_fifa.sort_index(ascending=False, inplace=True)
df_fifa.head()

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
Z. Ibrahimović,41236,36,Sweden,85,85,LA Galaxy,14.0,15.0,RS,"Mar 23, 2018",195.58,94.800728,€21M
Y. Carrasco,208418,24,Belgium,83,86,Dalian YiFang FC,33.0,20.0,LM,"Feb 26, 2018",154.94,73.028312,€73.4M
Y. Brahimi,184267,28,Algeria,85,85,FC Porto,39.0,28.0,LM,"Jul 22, 2014",175.26,66.224432,€78M
Willian,180403,29,Brazil,84,84,Chelsea,30.5,175.0,RW,"Aug 28, 2013",175.26,78.017824,€56.4M
William Carvalho,207566,26,Portugal,84,86,Real Betis,31.5,38.0,CDM,"Jul 13, 2018",187.96,83.007336,€68.5M


El método `DataFrame.sort_values` ordena el `DataFrame` en función de una o varias columnas. El siguiente código ordena ascendentemente en función del campo _Age_ y, en segunda instancia, descencentemente en función del campo _Overall_ (en realidad se hace al revés).

In [14]:
df_fifa.sort_values(['Age', 'Overall'], ascending=[True, False], inplace=True)
df_fifa.head(10)

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
K. Mbappé,231747,19,France,88,95,Paris Saint-Germain,81.0,100.0,RM,"Jul 1, 2018",154.94,73.028312,€166.1M
O. Dembélé,231443,21,France,83,92,FC Barcelona,40.0,155.0,RW,"Aug 28, 2017",154.94,67.131616,€90M
Gabriel Jesus,230666,21,Brazil,83,92,Manchester City,41.0,130.0,ST,"Aug 3, 2016",175.26,73.028312,€84.1M
L. Sané,222492,22,Germany,86,92,Manchester City,61.0,195.0,LW,"Aug 2, 2016",182.88,74.842680,€125.1M
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M
N. Süle,212190,22,Germany,84,90,FC Bayern München,36.5,84.0,CB,"Jul 1, 2017",195.58,97.068688,€67.5M
D. Sánchez,220793,22,Colombia,84,88,Tottenham Hotspur,34.0,105.0,RCB,"Aug 26, 2017",187.96,78.925008,€65.5M
D. Alli,211117,22,England,84,90,Tottenham Hotspur,42.5,115.0,LCM,"Feb 2, 2015",187.96,79.832192,€87.1M
A. Martial,211300,22,France,84,90,Manchester United,42.5,165.0,LW,"Sep 1, 2015",182.88,76.203456,€87.1M


<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: El parámetro `kind` permite determinar el algoritmo de ordenación cuando se ordena una sola columna. En este caso, es interesante saber que `mergesort` es un algoritmo de ordenación estable. Es decir, cuando ordena por un campo, para los elementos con un mismo valor de ese campo, preserva el orden relativo de los elementos anterior a la ordenación.
</div>

La función `rank()` ordena todas las columnas y devuelve un `DataFrame` con el orden correspondiente a cada fila según cada columna.

In [15]:
df_fifa.rank().head(20)

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
K. Mbappé,199.0,1.0,89.5,167.0,200.0,150.0,194.0,75.0,162.5,124.0,22.0,58.0,41.0
O. Dembélé,198.0,2.5,89.5,23.0,189.5,44.5,126.0,122.0,176.0,21.0,22.0,15.5,192.0
Gabriel Jesus,197.0,2.5,41.5,23.0,189.5,116.5,130.0,107.5,191.0,23.5,79.0,58.0,180.0
L. Sané,191.0,9.5,112.0,134.5,189.5,116.5,174.0,153.0,120.5,8.0,103.0,76.0,24.0
Marco Asensio,189.0,9.5,180.0,107.0,189.5,167.0,162.0,168.0,176.0,98.5,103.0,89.5,20.0
N. Süle,173.0,9.5,112.0,68.0,165.5,58.0,108.5,54.5,17.5,114.0,196.5,199.5,147.0
D. Sánchez,188.0,9.5,60.5,68.0,127.0,188.0,93.5,78.5,136.5,17.0,148.5,116.0,136.5
D. Alli,170.0,9.5,74.5,68.0,165.5,188.0,133.5,90.5,88.5,42.5,148.5,126.0,182.5
A. Martial,171.0,9.5,89.5,68.0,165.5,127.5,133.5,133.0,120.5,190.5,103.0,89.5,182.5


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section6"></a>
# <font color="#7F000E" size=5> 6. Operaciones sobre elementos </font>
<br>

En este caso, se utilizará el _DataFrame_ ejemplo, parecido al utilizado en la libreta anterior. 

In [16]:
ventas = [('Álvaro', 'Queso', 'Álvarez', 15.0, 22.5),
         ('Benito',  'Vino', 'Benitez', 10.0, 14.5),
         ('Fernando', 'Jamón', 'Fernández', 35, 50),
         ('Martín',  'Aceite', 'Martínez', 12, 20),
         ('Hernán', 'Azafrán', 'Hernández', 3, 5)]

columnas = ['Nombre', 'Producto', 'Apellido', 'Compra', 'Venta']
indice =['Tienda 1', 'Tienda 1', 'Tienda 2', 'Tienda 3','Tienda 3']

df = pd.DataFrame(ventas, index=indice, columns=columnas)
df['Localidad']='Albacete'
df

,Nombre,Producto,Apellido,Compra,Venta,Localidad
Tienda 1,Álvaro,Queso,Álvarez,15.0,22.5,Albacete
Tienda 1,Benito,Vino,Benitez,10.0,14.5,Albacete
Tienda 2,Fernando,Jamón,Fernández,35.0,50.0,Albacete
Tienda 3,Martín,Aceite,Martínez,12.0,20.0,Albacete
Tienda 3,Hernán,Azafrán,Hernández,3.0,5.0,Albacete


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section61"></a> 
## <font color="#7F000E">Operaciones básicas </font>

Es posible operar sobre las columnas del mismo modo que se hace en el caso de los `Series`.

El siguiente ejemplo añade una columna con el _IVA_ de los productos, calculada a partir de los valores en la columna _Precio_.

In [17]:
df_copia = df.copy()                             # Copia el original (por claridad en los ejemplos))
df_copia['IVA']= df_copia['Venta']*0.21          # Calcula y añade el iva
df_copia

,Nombre,Producto,Apellido,Compra,Venta,Localidad,IVA
Tienda 1,Álvaro,Queso,Álvarez,15.0,22.5,Albacete,4.725
Tienda 1,Benito,Vino,Benitez,10.0,14.5,Albacete,3.045
Tienda 2,Fernando,Jamón,Fernández,35.0,50.0,Albacete,10.500
Tienda 3,Martín,Aceite,Martínez,12.0,20.0,Albacete,4.200
Tienda 3,Hernán,Azafrán,Hernández,3.0,5.0,Albacete,1.050


En el siguiente ejemplo, se crea una columna mediante una suma de otras dos (se suman dos `Series`). 

In [18]:
df_copia['P.V.P'] = df_copia['Venta']+df_copia['IVA']
df_copia

,Nombre,Producto,Apellido,Compra,Venta,Localidad,IVA,P.V.P
Tienda 1,Álvaro,Queso,Álvarez,15.0,22.5,Albacete,4.725,27.225
Tienda 1,Benito,Vino,Benitez,10.0,14.5,Albacete,3.045,17.545
Tienda 2,Fernando,Jamón,Fernández,35.0,50.0,Albacete,10.500,60.500
Tienda 3,Martín,Aceite,Martínez,12.0,20.0,Albacete,4.200,24.200
Tienda 3,Hernán,Azafrán,Hernández,3.0,5.0,Albacete,1.050,6.050


También se pueden modificar las propias columnas, y utilizar otro tipo de operaciones, como por ejemplo, para Strings. 

In [19]:
df_copia['Nombre'] =  df_copia['Nombre']+" "+df_copia['Apellido'] # Junta nombre y apellido en 'Nombre'
del df_copia['Apellido']                                          # Borra la columna "apellido"
df_copia

,Nombre,Producto,Compra,Venta,Localidad,IVA,P.V.P
Tienda 1,Álvaro Álvarez,Queso,15.0,22.5,Albacete,4.725,27.225
Tienda 1,Benito Benitez,Vino,10.0,14.5,Albacete,3.045,17.545
Tienda 2,Fernando Fernández,Jamón,35.0,50.0,Albacete,10.500,60.500
Tienda 3,Martín Martínez,Aceite,12.0,20.0,Albacete,4.200,24.200
Tienda 3,Hernán Hernández,Azafrán,3.0,5.0,Albacete,1.050,6.050


<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Si es posible, se ha de operar así, ya que es el modo más eficiente. 
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section62"></a> 
## <font color="#7F000E">Operaciones de transformación sobre elementos</font>


### <font color="#7F000E" face="monospace">Series.map()</font>

La función `Series.map()` se puede aplicar sobre columnas individuales, a cada uno de los elementos, del mismo modo que se aplicaba sobre  _Series_.

In [20]:
df_copia = df.copy()            # Copia el original (por claridad en los ejemplos))

def IVA(x):                     # Crea la función que aplica el IVA
    return x*0.21


df_copia['IVA'] = df_copia['Venta'].map(IVA)
#df_copia['IVA'] = df_copia['Venta'].map(lambda pr: pr*0.21) 
df_copia

,Nombre,Producto,Apellido,Compra,Venta,Localidad,IVA
Tienda 1,Álvaro,Queso,Álvarez,15.0,22.5,Albacete,4.725
Tienda 1,Benito,Vino,Benitez,10.0,14.5,Albacete,3.045
Tienda 2,Fernando,Jamón,Fernández,35.0,50.0,Albacete,10.500
Tienda 3,Martín,Aceite,Martínez,12.0,20.0,Albacete,4.200
Tienda 3,Hernán,Azafrán,Hernández,3.0,5.0,Albacete,1.050


### <font color="#7F000E" face="monospace">Series.apply()</font>

Del mismo modo, puede  la función `Series.apply()` permite generar nuevas columnas. 

In [21]:
df_copia = df.copy()            # Copia el original (por claridad en los ejemplos))


df_copia['IVA'] = df_copia['Venta'].apply(IVA) 
#df_copia['IVA'] = df_copia['Venta'].apply(lambda pr: pr*0.21)  # Hace lo mismo, pero utilizando una función lambda

df_copia

,Nombre,Producto,Apellido,Compra,Venta,Localidad,IVA
Tienda 1,Álvaro,Queso,Álvarez,15.0,22.5,Albacete,4.725
Tienda 1,Benito,Vino,Benitez,10.0,14.5,Albacete,3.045
Tienda 2,Fernando,Jamón,Fernández,35.0,50.0,Albacete,10.500
Tienda 3,Martín,Aceite,Martínez,12.0,20.0,Albacete,4.200
Tienda 3,Hernán,Azafrán,Hernández,3.0,5.0,Albacete,1.050


### <font color="#7F000E" face="monospace">DataFrame.apply()</font>

La función `DataFrame.apply()`, acepta como  parámetro `axis`, que puede tomar los valores `index/0` o `columns/1`. 

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: El parámetro `axis=0` o `axis='index'` indica que los datos que se pasan a la función _usan el índice_ del `DataFrame`, es decir, se le pasa un `Series` por cada columna.
</div>


In [22]:
def descuento(x):                     # Crea la función que aplica el descuento
    return x*0.95

df_copia[['Compra','Venta']].apply(descuento, axis=0)

,Compra,Venta
Tienda 1,14.25,21.375
Tienda 1,9.50,13.775
Tienda 2,33.25,47.500
Tienda 3,11.40,19.000
Tienda 3,2.85,4.750


<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: El parámetro `axis=1` o `axis='columns'` indica que los datos que se pasan a la función usan _las columnas_ del `DataFrame` como índice, es decir, se le pasa un `Series` por cada fila.
</div>

Esto permite operar con varias columnas y obtener nuevas columnas como resultado. Este ejemplo construye un String a partir de los datos de tres columnas. 

In [23]:
df_precios = df[['Producto','Compra','Venta']]
print(df_precios)
print()

def anuncio(entrada):
    return '¡'+entrada['Producto']+' a '+str(entrada['Venta']-entrada['Compra']) + ' euros!'

print(df_precios.apply(anuncio, axis=1))

         Producto  Compra  Venta
Tienda 1    Queso    15.0   22.5
Tienda 1     Vino    10.0   14.5
Tienda 2    Jamón    35.0   50.0
Tienda 3   Aceite    12.0   20.0
Tienda 3  Azafrán     3.0    5.0

Tienda 1      ¡Queso a 7.5 euros!
Tienda 1       ¡Vino a 4.5 euros!
Tienda 2     ¡Jamón a 15.0 euros!
Tienda 3     ¡Aceite a 8.0 euros!
Tienda 3    ¡Azafrán a 2.0 euros!
dtype: object



### <font color="#7F000E" face="monospace"> applymap() </font>

La función `applymap` aplica la función de transformación sobre todos los elementos del _DataFrame_. En este ejemplo concreto, copiamos todas las columnas numéricas (tipo `np.number`) en el `DataFrame` y se aplica una función que convierte los valores a String con tres decimales.

In [24]:
import numpy as np

df_copia = df.select_dtypes(include=np.number)
df_copia = df_copia.applymap(lambda x: '{:.3f}'.format(x))
df_copia

,Compra,Venta
Tienda 1,15.000,22.500
Tienda 1,10.000,14.500
Tienda 2,35.000,50.000
Tienda 3,12.000,20.000
Tienda 3,3.000,5.000


### <font color="#7F000E" face="monospace"> transform() </font>

La función `transform()` permite llevar a cabo una transformación de los elementos de un `DataFrame`. Permite especificar qué columnas se transforman, y también aplicar varias transformaciones con una sola llamada. Como resultado, devuelve una estructura que contiene todas las transformaciones llevadas a cabo  ([documentación](http://pandas.pydata.org/pandas-docs/version/0.23/basics.html#transform-api)).

In [25]:
display(df)
df_copia = df.copy()
df_copia.transform({'Venta': [lambda p: p*1.21, lambda p: "{:.1f} euros".format(p*1.21)], 'Localidad':str.upper})

,Nombre,Producto,Apellido,Compra,Venta,Localidad
Tienda 1,Álvaro,Queso,Álvarez,15.0,22.5,Albacete
Tienda 1,Benito,Vino,Benitez,10.0,14.5,Albacete
Tienda 2,Fernando,Jamón,Fernández,35.0,50.0,Albacete
Tienda 3,Martín,Aceite,Martínez,12.0,20.0,Albacete
Tienda 3,Hernán,Azafrán,Hernández,3.0,5.0,Albacete


Venta             Localidad
         <lambda>    <lambda>     upper
Tienda 1   27.225  27.2 euros  ALBACETE
Tienda 1   17.545  17.5 euros  ALBACETE
Tienda 2   60.500  60.5 euros  ALBACETE
Tienda 3   24.200  24.2 euros  ALBACETE
Tienda 3    6.050   6.0 euros  ALBACETE

La función `assign`, que permite ___replicar___ un `DataFrame` con nuevas columnas, también permite la operación con funciones.

In [26]:
df_copia.assign(PVP=lambda p: p['Venta']*0.21, Margen=lambda p: p['Venta']-p['Compra'])

,Nombre,Producto,Apellido,Compra,Venta,Localidad,PVP,Margen
Tienda 1,Álvaro,Queso,Álvarez,15.0,22.5,Albacete,4.725,7.5
Tienda 1,Benito,Vino,Benitez,10.0,14.5,Albacete,3.045,4.5
Tienda 2,Fernando,Jamón,Fernández,35.0,50.0,Albacete,10.500,15.0
Tienda 3,Martín,Aceite,Martínez,12.0,20.0,Albacete,4.200,8.0
Tienda 3,Hernán,Azafrán,Hernández,3.0,5.0,Albacete,1.050,2.0


La función asociada con `aplymap()` se aplica a todos los elementos del DataFrame dado y, por lo tanto, el método applymap() se define solo para **DataFrames**. De manera similar, la función asociada con el método `apply()` se puede aplicar a todos los elementos de **DataFrame** o **Series**, y por lo tanto, el método apply() se define para los objetos Series y DataFrame. El método `map()` solo se puede definir para objetos **Series** en Pandas.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section7"></a>
# <font color="#7F000E" size=5> 7. Agregación </font>
<br>

<a id="section71"></a> 
## <font color="#7F000E">Estadísticos descriptivos </font>

Cuando las funciones para la obtención de medidas resumen datos estadísticos se aplican sobre un objeto `DataFrame`, es necesario especificar sobre qué eje se hacen mediante el parámetro `axis` que  puede tomar los valores `index/0` o `columns/1`. Un resumen de las funciones disponbiles puede encontrarse en la ([documentación](http://pandas.pydata.org/pandas-docs/version/0.23/basics.html#descriptive-statistics)).

El siguiente código hace la suma y media de las filas, es decir, calcula para cada columna. Puede especificarse que solamente lo haga con las numéricas, aunque este es el comportamiento por defecto en algunas funciones cuando no se pueden sumar las filas o columnas.

In [27]:
# Obtiene estadísticos por columnas
print(df_copia.sum(axis=0, numeric_only=True))
print()
print(df_copia.mean(axis=0,numeric_only=True))
print()
print(df_copia.max(axis=0, numeric_only=True))

Compra     75.0
Venta     112.0
dtype: float64

Compra    15.0
Venta     22.4
dtype: float64

Compra    35.0
Venta     50.0
dtype: float64


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="#7F000E"></i></font></a>
</div>

---

<a id="section72"></a> 

## <font color="#7F000E" > Agregación de datos: <font face="monospace">agg() y apply()</font></font>

### <font color="#7F000E" face="monospace"> agg()  </font>

Las funciones `aggregate()` o `agg()` (son la misma) permiten aplicar funciones de agregación a filas o columnas (también admite el parámetro `axis`. A diferencia de `apply()`, descrito anteriormente, permite aplicar varias funciones en una operación. Éstas pueden ser referidas por un nombre (String), identificador, o incluso ser funciones `lambda` ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html)).

Por ejemplo, el código siguiente aplica las funciones suma, media y rango de dos columnas (en realidad, se crea un `DataFrame` con dos columnas y se aplica la agregación). 

In [28]:
df_precios[['Compra','Venta']].agg(['sum', np.mean, lambda col: col.max()-col.min()], axis=0)

,Compra,Venta
sum,75.0,112.0
mean,15.0,22.4
<lambda>,32.0,45.0


Este código aplica dos funciones para cada fila. 

In [29]:
# Review: https://github.com/pandas-dev/pandas/issues/7186
# This has caused me huge frustration and I believe this should be updated to allow passing the same function and then providing the desired name of the output column. 
# I'm working with a custom aggregation function that takes an additional argument by using functool's partial or simply using multiple lambda functions. 
# I was hoping to avoid 6 separate named functions, but with the current method I have to do that, even though each function is only slightly different than the other. 
# The "workarounds" here don't save any time compared to just having separately defined functions that are all very similar.

#df_precios[['Compra','Venta']].agg(lambda p:p['Venta']- p['Compra'],lambda p:p['Venta']/p['Compra']-1,axis=1)
#df_precios.agg({Compra:lambda p:p['Venta']- p['Compra'],Venta:lambda p:p['Venta']/p['Compra']-1},axis=1)                                    

`agg()` permite que se especifique, mediante un diccionario, qué función o funciones se aplican a cada columna.

In [30]:
df_precios.agg({'Compra':np.min, 'Venta':[np.mean, np.sum]}, axis=0)

,Compra,Venta
amin,3.0,NaN
mean,NaN,22.4
sum,NaN,112.0


### <font color="#7F000E" face="monospace"> apply()  </font>

La función `DataFrame.apply()`, puede utilizarse también para hacer agregaciones. 

In [31]:
df_precios = df[['Compra','Venta']]
print(df_precios)
print()

def min_max(valores):
    return (np.min(valores),np.max(valores))

# Valor mínimo y máximo por cada producto
# print(df_precios.apply(min_max, axis=0))
print(df_precios.apply(min_max))                # Equivalente
#print(df_precios.apply(np.mean))                # Calcula la media

          Compra  Venta
Tienda 1    15.0   22.5
Tienda 1    10.0   14.5
Tienda 2    35.0   50.0
Tienda 3    12.0   20.0
Tienda 3     3.0    5.0

   Compra  Venta
0     3.0    5.0
1    35.0   50.0


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section73"></a> 

## <font color="#7F000E" > Agregación de datos mediante ventana: <font face="monospace">rolling()</font> y <font face="monospace">expanding()</font></font>
<br>

La función `rolling` permite hacer agregaciones mediante una ventana deslizante. El parámetro `window` determina el tamaño de  la ventana, y `min_periods` el número mínimo de observaciones necesario para obtener un valor.  

La siguiente celda lee un `DataFrame` con el nivel de `CO` en la ciudad de madrid en el año 2017. Como resultado de la aplicación del operador, en cada entrada, registra la media de las cuatro últimas. Como `min_periods=2`, en la segunda y tercera fila calcula la media con los datos disponibles. 

In [32]:
df_2017 = pd.read_csv('data/madrid_2017.csv',  parse_dates=['date'], index_col=['station','date']).sort_index()

display(df_2017['CO'].head(20))

# Muestra la media de las cuatro ultimas horas
df_2017['CO'].head(20).rolling(window=4, min_periods=2).mean()

station   date               
28079004  2017-01-01 01:00:00    0.6
          2017-01-01 02:00:00    0.6
          2017-01-01 03:00:00    0.5
          2017-01-01 04:00:00    0.5
          2017-01-01 05:00:00    0.4
          2017-01-01 06:00:00    0.4
          2017-01-01 07:00:00    0.4
          2017-01-01 08:00:00    0.6
          2017-01-01 09:00:00    0.4
          2017-01-01 10:00:00    0.4
          2017-01-01 11:00:00    0.5
          2017-01-01 12:00:00    0.6
          2017-01-01 13:00:00    0.5
          2017-01-01 14:00:00    0.4
          2017-01-01 15:00:00    0.3
          2017-01-01 16:00:00    0.2
          2017-01-01 17:00:00    0.2
          2017-01-01 18:00:00    0.3
          2017-01-01 19:00:00    0.3
          2017-01-01 20:00:00    0.5
Name: CO, dtype: float64

station   date               
28079004  2017-01-01 01:00:00         NaN
          2017-01-01 02:00:00    0.600000
          2017-01-01 03:00:00    0.566667
          2017-01-01 04:00:00    0.550000
          2017-01-01 05:00:00    0.500000
          2017-01-01 06:00:00    0.450000
          2017-01-01 07:00:00    0.425000
          2017-01-01 08:00:00    0.450000
          2017-01-01 09:00:00    0.450000
          2017-01-01 10:00:00    0.450000
          2017-01-01 11:00:00    0.475000
          2017-01-01 12:00:00    0.475000
          2017-01-01 13:00:00    0.500000
          2017-01-01 14:00:00    0.500000
          2017-01-01 15:00:00    0.450000
          2017-01-01 16:00:00    0.350000
          2017-01-01 17:00:00    0.275000
          2017-01-01 18:00:00    0.250000
          2017-01-01 19:00:00    0.250000
          2017-01-01 20:00:00    0.325000
Name: CO, dtype: float64

La función `expanding` considera todas las entradas de las filas anteriores. Por defecto toma un parámetro, que es el equivalente a `min_periods` en `rolling`.

In [33]:
df_2017['CO'].head(20).expanding(4).mean()

station   date               
28079004  2017-01-01 01:00:00         NaN
          2017-01-01 02:00:00         NaN
          2017-01-01 03:00:00         NaN
          2017-01-01 04:00:00    0.550000
          2017-01-01 05:00:00    0.520000
          2017-01-01 06:00:00    0.500000
          2017-01-01 07:00:00    0.485714
          2017-01-01 08:00:00    0.500000
          2017-01-01 09:00:00    0.488889
          2017-01-01 10:00:00    0.480000
          2017-01-01 11:00:00    0.481818
          2017-01-01 12:00:00    0.491667
          2017-01-01 13:00:00    0.492308
          2017-01-01 14:00:00    0.485714
          2017-01-01 15:00:00    0.473333
          2017-01-01 16:00:00    0.456250
          2017-01-01 17:00:00    0.441176
          2017-01-01 18:00:00    0.433333
          2017-01-01 19:00:00    0.426316
          2017-01-01 20:00:00    0.430000
Name: CO, dtype: float64

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section8"></a>
# <font color="#7F000E"> 8. Agrupamiento: <font face="monospace"> groupby()</font></font>
<br>

La función `groupby()` permite agrupar los datos del `DataFrame` según valores de su índice o columnas. Devuelve una estructura del tipo `DataFrameGroupBy`, que implementa estructuras de datos necesarias para que las operaciones sobre grupos se apliquen de manera eficiente.

In [34]:
df_fifa.head()

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
K. Mbappé,231747,19,France,88,95,Paris Saint-Germain,81.0,100.0,RM,"Jul 1, 2018",154.94,73.028312,€166.1M
O. Dembélé,231443,21,France,83,92,FC Barcelona,40.0,155.0,RW,"Aug 28, 2017",154.94,67.131616,€90M
Gabriel Jesus,230666,21,Brazil,83,92,Manchester City,41.0,130.0,ST,"Aug 3, 2016",175.26,73.028312,€84.1M
L. Sané,222492,22,Germany,86,92,Manchester City,61.0,195.0,LW,"Aug 2, 2016",182.88,74.842680,€125.1M
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M


La siguiente celda de código agrupa las entradas del conjunto de datos anterior en función del valor del campo `Nationality`.

In [35]:
# grupos_df = df_fifa.groupby(df['Nationality']); # Las dos formas son equivalentes. La primera permite entender mejor el 
grupos_df = df_fifa.groupby('Nationality');       # funcionamiento de la función. La segunda es más cómoda. 
print(type(grupos_df))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


Es posible ___extraer___ un `DataFrame` con los elementos de cada grupo en `DataFrameGroupBy`.

In [36]:
grupos_df.get_group('Spain').head()

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M
Saúl,208421,23,Spain,85,90,Atlético Madrid,50.5,77.0,RCM,"Jul 1, 2013",182.88,77.110640,€107.3M
Kepa,206585,23,Spain,83,91,Chelsea,28.5,84.0,GK,"Aug 8, 2018",185.42,84.821704,€58.4M
Suso,202651,24,Spain,83,86,Milan,33.0,115.0,RW,"Jan 17, 2015",175.26,69.853168,€58.6M
Isco,197781,26,Spain,88,91,Real Madrid,73.5,315.0,LW,"Jul 3, 2013",175.26,78.925008,€156.2M


También posible iterar sobre la estructura `DataFrameGroupBy` y obtener el `DataFrame` correspondiente a cada grupo.

In [37]:
print("Imprime el primer grupo. \n")
for grupo, df_grupo in df_fifa.groupby('Nationality'):
    print("Grupo: ",grupo)
    display(df_grupo.head())
    break  

Imprime el primer grupo. 

Grupo:  Algeria


,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
R. Mahrez,204485,27,Algeria,85,85,Manchester City,40.5,205.0,RW,"Jul 10, 2018",154.94,67.131616,€74.9M
Y. Brahimi,184267,28,Algeria,85,85,FC Porto,39.0,28.0,LM,"Jul 22, 2014",175.26,66.224432,€78M


La estructura `GroupBy.indices`, contiene un diccionario con las posiciones de las filas que corresponden a cada uno de los grupos. Otra estructura, `GroupBy.groups`, devuelve los índices con las filas correspondientes a cada grupo. 

In [38]:
print("Índices")
print(type(grupos_df.indices))

print("\nClaves")
print(grupos_df.indices.keys())

print("\nPosiciones")
print(grupos_df.indices['Spain'])

Índices
<class 'dict'>

Claves
dict_keys(['Algeria', 'Argentina', 'Armenia', 'Austria', 'Belgium', 'Bosnia Herzegovina', 'Brazil', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Denmark', 'Egypt', 'England', 'Finland', 'France', 'Gabon', 'Germany', 'Greece', 'Guinea', 'Italy', 'Korea Republic', 'Montenegro', 'Morocco', 'Netherlands', 'Poland', 'Portugal', 'Senegal', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Uruguay', 'Wales'])

Posiciones
[  4  17  25  38  64  71  76  78  81  84  88  93 102 103 111 116 120 123
 126 131 134 137 152 161 172 174 176 182 191]


In [39]:
print("Grupos")
print(type(grupos_df.groups))

print("\nClaves")
print(grupos_df.indices.keys())

print("\nEntradas")
grupos_df.groups['Spain']

Grupos
<class 'pandas.io.formats.printing.PrettyDict'>

Claves
dict_keys(['Algeria', 'Argentina', 'Armenia', 'Austria', 'Belgium', 'Bosnia Herzegovina', 'Brazil', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Denmark', 'Egypt', 'England', 'Finland', 'France', 'Gabon', 'Germany', 'Greece', 'Guinea', 'Italy', 'Korea Republic', 'Montenegro', 'Morocco', 'Netherlands', 'Poland', 'Portugal', 'Senegal', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Uruguay', 'Wales'])

Entradas


Index(['Marco Asensio', 'Saúl', 'Kepa', 'Suso', 'Isco', 'Koke', 'Carvajal',
       'Sergi Roberto', 'Manu Trigueros', 'Gerard Moreno', 'De Gea', 'Thiago',
       'Rodrigo', 'Lucas Vázquez', 'Azpilicueta', 'Illarramendi',
       'Nacho Fernández', 'Sergio Busquets', 'Jordi Alba', 'Parejo',
       'Diego Costa', 'Sergio Asenjo', 'Iago Aspas', 'Piqué', 'José Callejón',
       'Sergio Ramos', 'David Silva', 'Raúl Albiol', 'Iniesta'],
      dtype='object', name='Name')

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Esta nomenclatura es totalmente contraintuitiva.
</div>

La estructura `DataFrameGroupBy` implementa la muchas de las funciones que implementa un `DataFrame`, pero éstas se aplican de manera independiente a cada uno de los grupos. El resultado de la aplicación es un `DataFrame`.

In [70]:
grupos_df = df_fifa.groupby('Nationality')
grupos_df..mean().head()

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause,Value (%)
Name,,,,,,,,,,,,,,
K. Mbappé,231747,19,France,88,95,Paris Saint-Germain,81.0,100.0,RM,"Jul 1, 2018",154.94,73.028312,€166.1M,1.011969
O. Dembélé,231443,21,France,83,92,FC Barcelona,40.0,155.0,RW,"Aug 28, 2017",154.94,67.131616,€90M,0.499738
Gabriel Jesus,230666,21,Brazil,83,92,Manchester City,41.0,130.0,ST,"Aug 3, 2016",175.26,73.028312,€84.1M,0.512231
L. Sané,222492,22,Germany,86,92,Manchester City,61.0,195.0,LW,"Aug 2, 2016",182.88,74.842680,€125.1M,0.762100
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M,0.674646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M. Mandžukić,181783,32,Croatia,84,84,Juventus,25.0,160.0,ST,"Jul 1, 2015",190.50,84.821704,€41.3M,0.312336
Falcao,167397,32,Colombia,84,84,AS Monaco,25.0,115.0,RS,"Jul 1, 2013",154.94,72.121128,€47.5M,0.312336
S. Handanovič,162835,33,Slovenia,88,88,Inter,30.0,110.0,GK,"Jul 1, 2012",193.04,92.079176,€51M,0.374803


El acceso a columnas también se aplica de manera independiente a cada grupo, de manera que genera un objeto `SeriesGroupBy` (o `DataFrameGroupBy` si se accede a varias columnas), en el que los datos están agrupados con el mismo criterio que el `DataFrame`.

In [41]:
grupos_club = df_fifa.groupby(df_fifa['Nationality'])['Club'] # Equivalente
#grupos_club = df_fifa['Club'].groupby(df_fifa['Nationality']) # Equivalente
#rupos_club = df_fifa.groupby('Nationality')['Club']           # Equivalente
print(type(grupos_club))

<class 'pandas.core.groupby.generic.SeriesGroupBy'>


In [42]:
for grupo, serie_grupo in grupos_club:
    print(grupo,": ",len(serie_grupo),"\n")
    print(serie_grupo.head())
    break;                                                      # Procesa solamente la primera iteración

Algeria :  2 

Name
R. Mahrez     Manchester City
Y. Brahimi           FC Porto
Name: Club, dtype: object


Pueden utilizarse varias columnas para hacer la agrupación.

In [43]:
grupos_df = df_fifa.groupby(['Nationality', 'Club'])
print(list(grupos_df.groups.keys())[:5])
print()

print(grupos_df.get_group(('Spain','FC Barcelona')))

[('Algeria', 'FC Porto'), ('Algeria', 'Manchester City'), ('Argentina', 'Atalanta'), ('Argentina', 'FC Barcelona'), ('Argentina', 'Inter')]

                     ID  Age Nationality  Overall  Potential          Club  \
Name                                                                         
Sergi Roberto    199564   26       Spain       83         86  FC Barcelona   
Sergio Busquets  189511   29       Spain       89         89  FC Barcelona   
Jordi Alba       189332   29       Spain       87         87  FC Barcelona   
Piqué            152729   31       Spain       87         87  FC Barcelona   

                 Value (M)  Wage (K) Position       Joined  Height     Weight  \
Name                                                                            
Sergi Roberto         26.5     170.0       RB  Jul 1, 2013  154.94  68.038800   
Sergio Busquets       51.5     315.0      CDM  Sep 1, 2008  187.96  76.203456   
Jordi Alba            38.0     250.0       LB  Jul 1, 2012  170.18

Es posible agrupar los datos según el resultado de una función aplicada sobre el índice del `DataFrame`. La siguiente celda de código implementa una función que devuelve el continente en el que está cada país.

In [44]:
continente = dict.fromkeys(['Algeria', 'Egypt','Gabon','Guinea', 'Morocco', 'Senegal'],'África')
continente.update(dict.fromkeys(['Argentina', 'Brazil', 'Chile', 'Colombia', 
                                 'Costa Rica', 'Uruguay'],'América'))
continente.update(dict.fromkeys(['Armenia', 'Korea Republic'],'Asia'))
continente.update(dict.fromkeys(['Austria', 'Belgium', 'Bosnia Herzegovina', 'Croatia', 
                                 'Denmark', 'England', 'Finland', 'France', 'Germany', 
                                 'Greece', 'Italy', 'Montenegro', 'Netherlands', 'Poland', 
                                 'Portugal', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 
                                 'Sweden', 'Wales'],'Europa'))
def fun_continente(pais):
    return continente[pais]

fun_continente('Spain')

'Europa'

La siguiente celda de código agrupa las entradas en función del resultado de la función aplicada a cada uno de los valores del índice. Luego muestra, para cada grupo, el valor total de los jugadores. 

In [45]:
for grupo, data in df_fifa.set_index('Nationality').groupby(fun_continente):
    print(grupo+": Valor total: ",data['Value (M)'].sum())

América: Valor total:  2037.0
Asia: Valor total:  62.5
Europa: Valor total:  5476.2
África: Valor total:  428.5


En este caso, se puede obtener el mismo resultado accediendo directamente con el diccionario. 

In [46]:
for grupo, data in df_fifa.set_index('Nationality').groupby(continente):
    print(grupo+": Valor total: ",data['Value (M)'].sum())

América: Valor total:  2037.0
Asia: Valor total:  62.5
Europa: Valor total:  5476.2
África: Valor total:  428.5


La operación anterior se puede sintetizar.

In [47]:
df_fifa.set_index('Nationality').groupby(continente)['Value (M)'].sum()

América    2037.0
Asia         62.5
Europa     5476.2
África      428.5
Name: Value (M), dtype: float64

Indirectamente, es posible agrupar a partir de una función aplicada sobre las columnas. El siguiente código es equivalente al anterior, pero no se establece la columna _Nationality_ como índice. 

In [48]:
#for grupo, data in df_fifa.groupby(df_fifa['Nationality'].apply(fun_continente)):
#for grupo, data in df_fifa.groupby(df_fifa['Nationality'].map(fun_continente)):
for grupo, data in df_fifa.groupby(df_fifa['Nationality'].map(continente)):
    print(grupo+": Valor total: ",data['Value (M)'].sum())

América: Valor total:  2037.0
Asia: Valor total:  62.5
Europa: Valor total:  5476.2
África: Valor total:  428.5


Esta forma de trabajar  permite hacer la agrupación mediante una función aplicada sobre varias columnas. El siguiente código agrupa los jugadores en función de si han alcanzado su  máximo potencial o no.

In [49]:
def max_potential(player):
    return player['Potential']-player['Overall']<1

grupos_df = df_fifa.groupby(df_fifa.apply(max_potential, axis=1))
# Muestra los que no han alcanzado su máximo potencial.
grupos_df.get_group(False).head()

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause
Name,,,,,,,,,,,,,
K. Mbappé,231747,19,France,88,95,Paris Saint-Germain,81.0,100.0,RM,"Jul 1, 2018",154.94,73.028312,€166.1M
O. Dembélé,231443,21,France,83,92,FC Barcelona,40.0,155.0,RW,"Aug 28, 2017",154.94,67.131616,€90M
Gabriel Jesus,230666,21,Brazil,83,92,Manchester City,41.0,130.0,ST,"Aug 3, 2016",175.26,73.028312,€84.1M
L. Sané,222492,22,Germany,86,92,Manchester City,61.0,195.0,LW,"Aug 2, 2016",182.88,74.842680,€125.1M
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section81"></a> 
## <font color="#7F000E">Agregación: <font face="monospace">GroupBy.agg()</font></font>
<br>

Una de los usos más frecuentes de la agrupación es la agregación por grupos. La función `agg()` lleva a cabo la agrupación de manera independiente para cada grupo. El resultado de la misma es un `DataFrame`.

In [50]:
media_pais = df_fifa.groupby('Nationality').agg({'Value (M)': [np.sum, lambda pop: np.max(pop)], 
                                                 'Wage (K)':'mean'})
media_pais.head()

Value (M)               Wage (K)
                  sum <lambda_0>        mean
Nationality                                 
Algeria          79.5       40.5  116.500000
Argentina       502.5      110.5  202.666667
Armenia          25.5       25.5  145.000000
Austria          38.0       38.0  110.000000
Belgium         543.5      102.0  179.727273

En las últimas versiones de Pandas, se permite nombrar las columnas resultantes de las agregaciones. Existe una restricción, y es la necesidad de que los nombres sean compatibles con los identificadores en Python.

In [51]:
media_pais = df_fifa.groupby('Nationality').agg(Total_V=('Value (M)',np.sum), 
                                                Máximo_V=('Value (M)',np.max),
                                                Medio_W=('Wage (K)', np.mean))

media_pais.head()

,Total_V,Máximo_V,Medio_W
Nationality,,,
Algeria,79.5,40.5,116.500000
Argentina,502.5,110.5,202.666667
Armenia,25.5,25.5,145.000000
Austria,38.0,38.0,110.000000
Belgium,543.5,102.0,179.727273


No obstante, se pueden renombrar las columnas. 

In [52]:
# Este método, simple, todavía funciona, pero está obsoleto (deprecated) y dejará de funcionar. 
# media_pais = df_fifa.groupby('Nationality').agg({'Value (M)':{'Valor total':np.sum,
#                                                              'Valor máximo':np.max},
#                                                 'Wage (K)':{'Ganancia media':np.mean}})

media_pais = (df_fifa.groupby('Nationality').agg(
    Total_V=('Value (M)',np.sum), 
    Máximo_V=('Value (M)',np.max),
    Medio_W=('Wage (K)', np.mean)
).rename(columns = {
    'Total_V':'Valor total',
    'Máximo_V':'Valor máximo',
    'Medio_W':'Ganancia media'
}))

media_pais.head()

,Valor total,Valor máximo,Ganancia media
Nationality,,,
Algeria,79.5,40.5,116.500000
Argentina,502.5,110.5,202.666667
Armenia,25.5,25.5,145.000000
Austria,38.0,38.0,110.000000
Belgium,543.5,102.0,179.727273


Existe otro modo de llevar a cabo la agregación. Consiste en acceder a la columna determinada, y llevar a cabo la agregación sobre ella.

In [53]:
media_pais = df_fifa.groupby('Nationality')['Value (M)'].agg([np.max, np.min, 'mean'])
media_pais.head()

,amax,amin,mean
Nationality,,,
Algeria,40.5,39.0,39.750000
Argentina,110.5,28.5,55.833333
Armenia,25.5,25.5,25.500000
Austria,38.0,38.0,38.000000
Belgium,102.0,22.0,49.409091


In [71]:
# Esta forma funciona, pero está también obsoleta (deprecated)
# media_pais = df_fifa.groupby('Nationality')['Value (M)'].agg({'Valor máximo':np.max, 'Valor medio':np.mean})

SpecificationError: nested renamer is not supported

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section82"></a> 
## <font color="#7F000E">Transformación: <font face="monospace">GroupBy.transform()</font> </font>
<br>

`GroupBy.transform` produce una transformación para cada columna del grupo, y devuelve un `Series` o `DataFrame` con las mismas filas que el `DataFrame` original, y los nuevos valores. Como parámetro acepta una función que actúa sobre el `Series`. Por tanto, dicha función puede devolver o bien un escalar o bien otro `Series`.

La siguiente celda devuelve un `DataFrame` en el que, el valor para cada jugador, corresponde con la media de la columna para el grupo al que pertenece.

In [72]:
df_fifa.groupby('Nationality')['Value (M)', 'Overall'].transform(lambda s: s.mean()).head()
#df_fifa.groupby('Nationality')['Value (M)', 'Overall'].transform('mean').head() # Equivalente

,Value (M),Overall
Name,,
K. Mbappé,43.454545,85.090909
O. Dembélé,43.454545,85.090909
Gabriel Jesus,35.483333,84.966667
L. Sané,39.071429,85.142857
Marco Asensio,37.965517,85.310345


En esta celda, para cada jugador, se representa el valor original menos la media para su grupo.

In [74]:
df_fifa.groupby('Nationality')['Value (M)', 'Overall'].transform(lambda s: s-s.mean()).head()

,Value (M),Overall
Name,,
K. Mbappé,37.545455,2.909091
O. Dembélé,-3.454545,-2.090909
Gabriel Jesus,5.516667,-1.966667
L. Sané,21.928571,0.857143
Marco Asensio,16.034483,-0.310345


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section83"></a> 
## <font color="#7F000E"><font face="monospace">GroupBy.apply()</font> </font>
<br>

Permite aplicar una función a cada uno de los grupos. La función tiene que tomar como entrada un `DataFrame` y puede devolver un escalar, un `Series`, o un `DataFrame`. Dependiendo de la salida de la función, `GroupBy.apply` devuelve un tipo u otro de estructura.  `GroupBy.apply` permite llevar a cabo tanto agregaciones como transformaciones. Por ejemplo, si la función devuelve un escalar, `GroupBy.apply` devuelve una serie con el valor correspondiente a cada grupo.

In [57]:
#len(df_fifa)
df_fifa.groupby('Nationality').apply(lambda data: len(data)).head()

Nationality
Algeria       2
Argentina     9
Armenia       1
Austria       1
Belgium      11
dtype: int64

Si la salida es un `Series`, depende de si el índice de ésta corresponde al índice o las columnas. En este caso, por ejemplo,  devuelve la media para cada columna.  Por tanto, el código devuelve la media para cada columna en cada grupo.

In [75]:
# df_fifa.mean()
df_fifa.groupby('Nationality')['Overall','Potential'].apply(lambda data: data.mean()).head()

,Overall,Potential
Nationality,,
Algeria,85.000000,85.000000
Argentina,87.111111,88.000000
Armenia,83.000000,83.000000
Austria,85.000000,87.000000
Belgium,86.818182,87.545455


Si la salida de la función es un `Series` cuyo índice es el índice del `DataFrame`, `GroupBy.apply()` devuelve un `Series` del tamaño del original. Esta funcionalidad se suele utilizar para calcular valores que dependen del grupo concreto. Por ejemplo, la siguiente celda crea una columna con el porcentaje que supone el valor de un jugador en el valor total de los jugadores.

In [59]:
def pct_val_player(data):
    return 100*data['Value (M)']/data['Value (M)'].sum()
    
pct_val_player(df_fifa).head()

Name
K. Mbappé        1.011969
O. Dembélé       0.499738
Gabriel Jesus    0.512231
L. Sané          0.762100
Marco Asensio    0.674646
Name: Value (M), dtype: float64

Al utilizarla con `GroupBy.apply`, para cada jugador representa el porcentaje de su valor en el valor total de los jugadores de su país. El `Series` resultante tiene un índice de dos niveles (se verá en el siguiente tutorial).

In [60]:
df_fifa.groupby('Nationality').apply(pct_val_player)

Nationality  Name      
Algeria      R. Mahrez      50.943396
             Y. Brahimi     49.056604
Argentina    P. Dybala      17.711443
             M. Icardi      12.835821
             S. Agüero      12.835821
                              ...    
Uruguay      J. Giménez     15.596330
             L. Suárez      36.697248
             E. Cavani      27.522936
             D. Godín       20.183486
Wales        G. Bale       100.000000
Name: Value (M), Length: 200, dtype: float64

En este ejemplo, la función devuelve el máximo y el mínimo para cada una de las columnas del `DataFrame`.

In [61]:
def max_min(data):
    return data.select_dtypes(include=np.number).agg([np.min, np.max])

max_min(data)

,ID,Age,Overall,Potential,Value (M),Wage (K),Height,Weight
amin,177509,23,83,83,21.0,28.0,154.94,63.956472
amax,220971,31,88,90,69.5,265.0,187.96,93.893544


Al utilizarla con `GroupBy.apply`, trata cada grupo por separado. El `DataFrame` resultante tiene un índice de dos niveles.

In [62]:
df_fifa.groupby('Nationality').apply(max_min).head(10)

ID  Age  Overall  Potential  Value (M)  Wage (K)  \
Nationality                                                              
Algeria     amin  184267   27       85         85       39.0      28.0   
            amax  204485   28       85         85       40.5     205.0   
Argentina   amin  143076   24       84         84       28.5      31.0   
            amax  211110   31       94         94      110.5     565.0   
Armenia     amin  192883   29       83         83       25.5     145.0   
            amax  192883   29       83         83       25.5     145.0   
Austria     amin  197445   26       85         87       38.0     110.0   
            amax  197445   26       85         87       38.0     110.0   
Belgium     amin  139720   24       83         84       22.0      20.0   
            amax  208418   32       91         92      102.0     355.0   

                  Height     Weight  
Nationality                          
Algeria     amin  154.94  66.224432  
            amax  175.26  67.131616  
Argentina   amin  154.94  68.038800  
            amax  185.42  88.904032  
Armenia     amin  154.94  74.842680  
            amax  154.94  74.842680  
Austria     amin  154.94  76.203456  
            amax  154.94  76.203456  
Belgium     amin  154.94  60.781328  
            amax  198.12  96.161504

En este último ejemplo, similar al visto anteriormente,  la función transforma el `DataFrame` añadiendo una columna con el porcentaje del valor total que supone cada jugador.

In [63]:
def pct_val_player(data):
    data['Value (%)'] = 100*data['Value (M)']/data['Value (M)'].sum()
    return data
    
pct_val_player(df_fifa).head()

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause,Value (%)
Name,,,,,,,,,,,,,,
K. Mbappé,231747,19,France,88,95,Paris Saint-Germain,81.0,100.0,RM,"Jul 1, 2018",154.94,73.028312,€166.1M,1.011969
O. Dembélé,231443,21,France,83,92,FC Barcelona,40.0,155.0,RW,"Aug 28, 2017",154.94,67.131616,€90M,0.499738
Gabriel Jesus,230666,21,Brazil,83,92,Manchester City,41.0,130.0,ST,"Aug 3, 2016",175.26,73.028312,€84.1M,0.512231
L. Sané,222492,22,Germany,86,92,Manchester City,61.0,195.0,LW,"Aug 2, 2016",182.88,74.842680,€125.1M,0.762100
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M,0.674646


Al aplicar esta función al grupo, añade el porcentaje del valor que supone el jugador dentro del total de jugadores de su país al `DataFrame`.

In [64]:
df_fifa.groupby('Nationality').apply(pct_val_player)

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause,Value (%)
Name,,,,,,,,,,,,,,
K. Mbappé,231747,19,France,88,95,Paris Saint-Germain,81.0,100.0,RM,"Jul 1, 2018",154.94,73.028312,€166.1M,8.472803
O. Dembélé,231443,21,France,83,92,FC Barcelona,40.0,155.0,RW,"Aug 28, 2017",154.94,67.131616,€90M,4.184100
Gabriel Jesus,230666,21,Brazil,83,92,Manchester City,41.0,130.0,ST,"Aug 3, 2016",175.26,73.028312,€84.1M,3.851574
L. Sané,222492,22,Germany,86,92,Manchester City,61.0,195.0,LW,"Aug 2, 2016",182.88,74.842680,€125.1M,7.434491
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M,4.904632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pepe,120533,35,Portugal,85,85,Beşiktaş JK,9.0,57.0,RCB,"Jul 4, 2017",187.96,81.192968,€17.1M,2.898551
Naldo,171919,35,Brazil,85,85,FC Schalke 04,9.0,38.0,CB,"Jul 1, 2016",198.12,91.171992,€15.3M,0.845467
Z. Ibrahimović,41236,36,Sweden,85,85,LA Galaxy,14.0,15.0,RS,"Mar 23, 2018",195.58,94.800728,€21M,100.000000


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>

---

<a id="section84"></a> 
## <font color="#7F000E">Filtrado: <font face="monospace">GroupBy.filter()</font> </font>
<br>

La función `filter` también permite filtrar grupos mediante alguna condición. Como resultado devuelve un `DataFrame` con los elementos correspondientes a los grupos que cumplen la condición.

In [65]:
#df_fifa.groupby('Nationality').filter(lambda x: len(x)>=15)             # Paises con más de 15 jugadores
df_fifa.groupby('Nationality').filter(lambda x: x['Overall'].mean()>85)  # Filtra por la valoración media de los jugadores

,ID,Age,Nationality,Overall,Potential,Club,Value (M),Wage (K),Position,Joined,Height,Weight,Release Clause,Value (%)
Name,,,,,,,,,,,,,,
K. Mbappé,231747,19,France,88,95,Paris Saint-Germain,81.0,100.0,RM,"Jul 1, 2018",154.94,73.028312,€166.1M,1.011969
O. Dembélé,231443,21,France,83,92,FC Barcelona,40.0,155.0,RW,"Aug 28, 2017",154.94,67.131616,€90M,0.499738
L. Sané,222492,22,Germany,86,92,Manchester City,61.0,195.0,LW,"Aug 2, 2016",182.88,74.842680,€125.1M,0.762100
Marco Asensio,220834,22,Spain,85,92,Real Madrid,54.0,215.0,RW,"Jul 1, 2015",182.88,76.203456,€121.5M,0.674646
N. Süle,212190,22,Germany,84,90,FC Bayern München,36.5,84.0,CB,"Jul 1, 2017",195.58,97.068688,€67.5M,0.456011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Iniesta,41,34,Spain,86,86,Vissel Kobe,21.5,21.0,LF,"Jul 16, 2018",170.18,68.038800,€26.9M,0.268609
Quaresma,20775,34,Portugal,84,84,Beşiktaş JK,15.5,80.0,RM,"Jul 22, 2015",175.26,67.131616,€29.5M,0.193648
Pepe,120533,35,Portugal,85,85,Beşiktaş JK,9.0,57.0,RCB,"Jul 4, 2017",187.96,81.192968,€17.1M,0.112441


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#7F000E"></i></font></a>
</div>


---

<a id="section86"></a> 
## <font color="#7F000E">Eficiencia </font>
<br>

Aunque la agregación por grupos se puede llevar a cabo de otros modos, el uso de una estructura `GroupBy` permite hacerlo de manera más eficiente.  La siguiente celda obtiene el valor (`Overall`) medio de los jugadores de cada club sin utilizar la estructura `GroupBy`.

Con el fin de que se aprecie la diferencia, se utilizarán 1000 registros del conjunto anterior.

In [76]:
df_fifa_all = pd.read_csv('./data/fifa19.csv', index_col=0).set_index('Name')
df_fifa_all = df_fifa_all.iloc[:1000]

In [77]:
%%timeit -n 10
for club in df_fifa_all['Club'].unique():
    avg = np.average(df_fifa_all[df_fifa_all['Club']==club]['Overall'])

120 ms ± 4.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


La siguiente celda hace la misma operación, pero iterando dobre una estructura `GroupBy`.

In [78]:
%%timeit -n 10
for grupo, frame in df_fifa_all.groupby('Club'):
    avg = np.average(frame['Overall'])

34.4 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Por último, en esta celda se lleva a cabo el mismo cálculo, pero sin iterar. 

In [79]:
%%timeit -n 10
avg = df_fifa_all.groupby('Club')['Overall'].mean()

2.13 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>